In [1]:

#Importing the necessary libraries

import os
import numpy as np
import random
import tensorflow
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import zipfile

In [19]:
# base_dir = os.getcwd()
# print(base_dir)
base_dir = os.getcwd()

train_dir = os.path.join(base_dir, 'train')
# Directory with our training cat/dog pictures
train_zero_dir = os.path.join(train_dir, 'zero')
train_one_dir = os.path.join(train_dir, 'one')
train_two_dir = os.path.join(train_dir, 'two')
train_three_dir = os.path.join(train_dir, 'three')
train_four_dir = os.path.join(train_dir, 'four')

validation_dir = os.path.join(base_dir)
# Directory with our validation cat/dog pictures
validation_dir = os.path.join(base_dir, 'test')
#validation_dogs_dir = os.path.join(validation_dir, 'dogs')

print('total training zero images :', len(os.listdir(train_zero_dir)))
print('total training one images :', len(os.listdir(train_one_dir)))
print('total training two images :', len(os.listdir(train_two_dir)))
print('total training three images :', len(os.listdir(train_three_dir)))
print('total training four images :', len(os.listdir(train_four_dir)))
# print('total training one images :', len(os.listdir(train_one_dir)))
print('total validation images :', len(os.listdir(validation_dir)))
#print('total validation dog images :', len(os.listdir(validation_dogs_dir)))

total training zero images : 231
total training one images : 17
total training two images : 39
total training three images : 5
total training four images : 8
total validation images : 180


In [11]:
# print(os.getcwd()+'\inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5//')
local_weights_file = os.path.join('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

pre_trained_model = InceptionV3(input_shape = (299, 299, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

255[0][0]             
__________________________________________________________________________________________________
conv2d_257 (Conv2D)             (None, 17, 17, 192)  147456      average_pooling2d_24[0][0]       
__________________________________________________________________________________________________
batch_normalization_248 (BatchN (None, 17, 17, 192)  576         conv2d_248[0][0]                 
__________________________________________________________________________________________________
batch_normalization_251 (BatchN (None, 17, 17, 192)  576         conv2d_251[0][0]                 
__________________________________________________________________________________________________
batch_normalization_256 (BatchN (None, 17, 17, 192)  576         conv2d_256[0][0]                 
__________________________________________________________________________________________________
batch_normalization_257 (BatchN (None, 17, 17, 192)  576         conv2d_257[0][0]     

In [12]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer taken from InceptionV3 output shape: ', last_layer.output_shape)

last_output = last_layer.output
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (5, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

last layer taken from InceptionV3 output shape:  (None, 17, 17, 768)


In [15]:
model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [16]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [20]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'categorical', 
                                                    target_size = (299, 299)) 


Found 300 images belonging to 5 classes.


In [21]:
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'categorical', 
                                                          target_size = (299, 299))

Found 0 images belonging to 0 classes.


In [24]:
history = model.fit_generator(
            train_generator,
            #validation_data = validation_generator,
            steps_per_epoch = 15,
            epochs = 2,
            #validation_steps = 50,
            verbose = 2)

Epoch 1/2
15/15 - 119s - loss: 1.6036 - acc: 0.7100
Epoch 2/2
15/15 - 104s - loss: 1.6787 - acc: 0.6500
